In [1]:
# import
import torch
from tqdm import tqdm
from transformers import BertConfig, BertForPreTraining
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil
from icecream import ic
from tqdm import tqdm_notebook as tqdm 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

from torch import nn

from transformers import BertModel
import os
os.environ["CUDA_VISIBLE_DEVICES"]="9"


In [2]:
dic = {}

with open("../BERT-base_mecab-ipadic-bpe-32k/vocab.txt",  "r", encoding="utf-8" ) as f:
    vocab = f.read()
    for id, word in enumerate(vocab.split('\n')):
        dic[word] = id
        
text = "[CLS] 私 は 犬 が 好き 。 [SEP]"

x = [dic[w] for w in text.split()]

In [3]:
x = torch.LongTensor(x).unsqueeze(0)



In [4]:
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")

In [5]:
a = bert(x)

In [6]:
a[0].shape

torch.Size([1, 8, 768])

In [7]:
a[0][0][3]

tensor([-1.2636e-01, -1.2734e-01,  4.1125e-01, -3.7946e-02,  3.5671e-01,
         5.9472e-01, -1.2080e-02, -2.0193e-01,  6.1377e-01,  2.1312e-01,
        -7.4529e-02,  2.0608e-02, -4.5949e-01,  2.5509e-01, -2.4717e-01,
        -9.4191e-02,  1.1410e-01, -2.2248e-01,  3.5165e-01,  4.6572e-01,
        -3.8768e-01, -8.5532e-02,  2.5526e-01,  2.6166e-01, -3.1824e-01,
         8.1794e-02, -2.4993e-01,  2.9269e-01,  7.9955e-01,  3.5923e-01,
        -1.2338e-01,  6.5812e-03,  3.8730e-01,  2.3091e-01,  1.4636e-01,
         4.6668e-01,  2.6290e-01,  1.6372e-01,  6.9316e-02,  1.4958e-01,
        -5.9775e-01,  2.1966e-01, -1.5152e-01, -7.7528e-03,  2.8760e-02,
         8.2232e-01, -2.1874e-01,  4.2410e-02,  8.8731e-01,  9.9789e-02,
        -6.2738e-01, -4.2279e-01,  9.3123e-03, -3.0431e-01, -2.4775e-01,
        -9.0133e-02,  1.4442e+00,  2.2802e-01,  4.4458e-01, -3.1874e-01,
        -6.9717e-01,  6.2764e-01, -3.7871e-02, -6.9396e-02,  1.3187e-01,
        -1.4630e-01, -9.5384e-02, -4.0134e-02, -1.1

In [8]:
from transformers import BertJapaneseTokenizer

tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')


In [9]:
tknz.tokenize("私は犬が好き")

['私', 'は', '犬', 'が', '好き']

# Masked Language Model

In [10]:
# ids = tknz.encode("私は[MASK]が好き")

In [11]:
# ids

In [12]:
# from transformers import BertForMaskedLM

# model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')

In [13]:
# a = model(torch.LongTensor(ids).unsqueeze(0))

In [14]:
# mskpos = 3
# b=torch.topk(a[0][0][mskpos], k=5)
# tknz.convert_ids_to_tokens(b[1])

In [15]:
x_train, x_test = [],[]
y_train, y_test = [],[]


In [16]:
train_df = pd.read_csv("../data/train_twitterJSA_data.csv")

test_df = pd.read_csv("../data/test_twitterJSA_data.csv")

In [17]:
# y_train = train_df["label"].tolist()
# y_test = test_df["label"].tolist()

In [18]:
for ind in tqdm(range(len(train_df))):
#     print(ind)
    
    if type(train_df["text"].iloc[ind]  ) != str:
        continue
    tid = tknz.encode(train_df["text"].iloc[ind]) 
    
    if (len(tid)>512):
        tid = tid[:512]
    x_train.append(tid)
    y_train.append(train_df["label"].iloc[ind])
#     break

In [19]:
# net(x_train[1])

# net(torch.LongTensor(x_train[1]).unsqueeze(0).to(device))

In [20]:
for i in range(len(x_train)):
    if (len(x_train[i])) < 2:
        print(x_train[i])


In [21]:
for ind in tqdm(range(len(test_df))):
    
    if type(test_df["text"].iloc[ind] ) != str:
        continue
    
    tid = tknz.encode(test_df["text"].iloc[ind]) 
    if (len(tid)>512):
        tid = tid[:512]
    x_test.append(tid)
    y_test.append(test_df["label"].iloc[ind])
#     break

In [22]:
class DocCls(nn.Module):
    def __init__(self, bert):
        super(DocCls, self).__init__()
        self.bert = bert
        self.cls = nn.Linear(768, 5)
    
    def forward(self, x):
        bout = self.bert(x)
        bs = len(bout[0])
        h0 = [bout[0][i][0] for i in range(bs)]
        h0 = torch.stack(h0, dim=0)
        return self.cls(h0)
        
    

In [23]:

# device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [24]:
from torch import optim
net = DocCls(bert)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

## 学習

In [25]:
net.train()

for ep in tqdm(range(10)):
    lossK=0.0
    for i in range(len(x_train)):
        x= torch.LongTensor(x_train[i]).unsqueeze(0).to(device)
        dummy_y  =[]
        added_flg = 0
        
        # 同率１位は、先のほうにする。
        for index, yy in enumerate(y_train[i][1:-1].split(",")):
            if "1" in yy and added_flg == 0:
                dummy_y.append(index)
                added_flg = 1
            
        y = torch.LongTensor(dummy_y).to(device)
        out = net(x)
#         print("out:", out.size(), "y:", y.size())
        
        loss = criterion(out, y)
        lossK += loss.item()
        
        if (i % 2000 == 0 ):
            print(ep, i, lossK)
            lossK = 0.0
        elif (i>50000):
            break
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    outfile = "doccls - "+ str(ep) + ".model"
            

0 0 1.273510456085205
0 2000 2098.084459245205
0 4000 1830.7071210741997
0 6000 1706.9611879587173
0 8000 1647.992381453514
0 10000 1618.5863126516342
0 12000 1540.898620545864
0 14000 1501.5223188996315
0 16000 1420.421553492546
0 18000 1472.0156088471413
0 20000 1402.6514811515808
0 22000 1439.4924801588058
0 24000 1360.3003531694412
0 26000 1342.4875227808952
0 28000 1369.2085596323013
0 30000 1337.4415910243988
0 32000 1331.3727125525475
0 34000 1384.6455251574516
0 36000 1263.418677330017
0 38000 1366.0616002082825
0 40000 1337.9645774364471
0 42000 1222.2548843622208
0 44000 1288.8126438856125
0 46000 1301.2395622730255
1 2000 1256.0370894670486
1 4000 1268.7430146932602
1 6000 1297.8115581274033
1 8000 1255.544875562191
1 10000 1284.3062624931335
1 12000 1207.629076719284
1 14000 1207.10584461689
1 16000 1169.90454351902
1 18000 1221.1940683722496
1 20000 1158.1328610181808
1 22000 1187.691885650158
1 24000 1116.101574063301
1 26000 1118.096886396408
1 28000 1130.0950382351875
1

In [26]:
torch.save(net.state_dict(), outfile)




## 評価

In [ ]:
real_data_num, ok =0, 0
net.eval()

with torch.no_grad():
    for i in range(len(x_test)):
        x = torch.LongTensor(x_test[i]).unsqueeze(0).to(device)
        ans =  net(x)
        ans1 = torch.argmax(ans,dim=1).item()
        
        if (ans1 == np.argmax( [int(a) for a in y_test[i][1:-1].split(",")])+1):
            ok += 1
        real_data_num += 1
print(ok, real_data_num, ok/real_data_num)


In [40]:
torch.argmax(y_test[0],dim=1).item()

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [47]:
# np.argmax(y_test[3])
type(y_test[3])

str

5

In [72]:
y_test[i]

'[0, 0, 0, 0, 1]'

In [ ]:
torch.argmax(ans,dim=1).item()

In [67]:
[int(a) for a in  ]

[0, 1, 4, 9, 16]

In [63]:
[i**2 for i in range(5)]


[0, 1, 4, 9, 16]